# Covid-19 Data Analysis

We'll be exploring the dataset from https://covid.ourworldindata.org

1. Understanding Our Data
2. Covid 19 World Analysis - Most Recent Info
3. World Covid-19 Growth Over Time
4. Confirmed Cases in Each Country
5. Mapping Covid Around the World
6. Evolution of Confirmed Cases PM
7. Comparing Continents & Countries
8. Preparing Dataset for Flourish Bar Chart Race
9. Flourish - Making a Covid Bar Chart Race

# 1. Understanding Our Data

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns

url='https://covid.ourworldindata.org/data/owid-covid-data.csv'
df = pd.read_csv(url)
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,ABW,North America,Aruba,2020-03-13,2.0,2.0,NaN,NaN,0.0,NaN,18.733,18.733,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
1,ABW,North America,Aruba,2020-03-19,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
2,ABW,North America,Aruba,2020-03-20,4.0,2.0,0.286,NaN,0.0,0.0,37.465,18.733,2.676,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
3,ABW,North America,Aruba,2020-03-21,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
4,ABW,North America,Aruba,2020-03-22,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN


In [67]:
df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,ABW,North America,Aruba,2020-03-13,2.0,2.0,NaN,NaN,0.0,NaN,18.733,18.733,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
1,ABW,North America,Aruba,2020-03-19,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
2,ABW,North America,Aruba,2020-03-20,4.0,2.0,0.286,NaN,0.0,0.0,37.465,18.733,2.676,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
3,ABW,North America,Aruba,2020-03-21,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
4,ABW,North America,Aruba,2020-03-22,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52005,NaN,NaN,International,2020-10-19,696.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52006,NaN,NaN,International,2020-10-20,696.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52007,NaN,NaN,International,2020-10-21,696.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52008,NaN,NaN,International,2020-10-22,696.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
df[df['location'] == 'Trinidad and Tobago']

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
46519,TTO,North America,Trinidad and Tobago,2020-03-13,1.0,1.0,NaN,NaN,0.0,NaN,0.715,0.715,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.37,1399491.0,266.886,36.2,10.014,5.819,28763.071,NaN,228.467,10.97,NaN,NaN,89.443,3.0,73.51,0.784
46520,TTO,North America,Trinidad and Tobago,2020-03-14,2.0,1.0,NaN,NaN,0.0,NaN,1.429,0.715,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.48,1399491.0,266.886,36.2,10.014,5.819,28763.071,NaN,228.467,10.97,NaN,NaN,89.443,3.0,73.51,0.784
46521,TTO,North America,Trinidad and Tobago,2020-03-15,2.0,0.0,NaN,NaN,0.0,NaN,1.429,0.000,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.48,1399491.0,266.886,36.2,10.014,5.819,28763.071,NaN,228.467,10.97,NaN,NaN,89.443,3.0,73.51,0.784
46522,TTO,North America,Trinidad and Tobago,2020-03-16,4.0,2.0,NaN,NaN,0.0,NaN,2.858,1.429,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.37,1399491.0,266.886,36.2,10.014,5.819,28763.071,NaN,228.467,10.97,NaN,NaN,89.443,3.0,73.51,0.784
46523,TTO,North America,Trinidad and Tobago,2020-03-17,5.0,1.0,NaN,NaN,0.0,NaN,3.573,0.715,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.07,1399491.0,266.886,36.2,10.014,5.819,28763.071,NaN,228.467,10.97,NaN,NaN,89.443,3.0,73.51,0.784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46739,TTO,North America,Trinidad and Tobago,2020-10-19,5297.0,16.0,28.000,96.0,1.0,0.857,3784.948,11.433,20.007,68.596,0.715,0.612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.78,1399491.0,266.886,36.2,10.014,5.819,28763.071,NaN,228.467,10.97,NaN,NaN,89.443,3.0,73.51,0.784
46740,TTO,North America,Trinidad and Tobago,2020-10-20,5298.0,1.0,26.000,97.0,1.0,0.714,3785.662,0.715,18.578,69.311,0.715,0.510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.78,1399491.0,266.886,36.2,10.014,5.819,28763.071,NaN,228.467,10.97,NaN,NaN,89.443,3.0,73.51,0.784
46741,TTO,North America,Trinidad and Tobago,2020-10-21,5333.0,35.0,29.429,98.0,1.0,0.714,3810.671,25.009,21.028,70.025,0.715,0.510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.78,1399491.0,266.886,36.2,10.014,5.819,28763.071,NaN,228.467,10.97,NaN,NaN,89.443,3.0,73.51,0.784
46742,TTO,North America,Trinidad and Tobago,2020-10-22,5392.0,59.0,34.000,101.0,3.0,1.143,3852.829,42.158,24.295,72.169,2.144,0.817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.78,1399491.0,266.886,36.2,10.014,5.819,28763.071,NaN,228.467,10.97,NaN,NaN,89.443,3.0,73.51,0.784


#### Let's get a quick summary of our missing data

In [69]:
# We use a list comprehension to perform this
# We use .isna() to find NaN values, then use the .mean function to get the average
# occurance and then multiple by 100 to convert to a percentage
NAN = [(c, df[c].isna().mean()*100) for c in df]

# We name the columns appropriately 
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN

,column_name,percentage
0,iso_code,0.572967
1,continent,1.145933
2,location,0.000000
3,date,0.000000
4,total_cases,6.935205
5,new_cases,1.697750
6,new_cases_smoothed,3.212844
7,total_deaths,23.941550
8,new_deaths,1.697750
9,new_deaths_smoothed,3.212844


#### Let's check to see how our location names look

In [70]:
print(df['location'].unique())
print(len(df['location'].unique()))

['Aruba' 'Afghanistan' 'Angola' 'Anguilla' 'Albania' 'Andorra'
 'United Arab Emirates' 'Argentina' 'Armenia' 'Antigua and Barbuda'
 'Australia' 'Austria' 'Azerbaijan' 'Burundi' 'Belgium' 'Benin'
 'Bonaire Sint Eustatius and Saba' 'Burkina Faso' 'Bangladesh' 'Bulgaria'
 'Bahrain' 'Bahamas' 'Bosnia and Herzegovina' 'Belarus' 'Belize' 'Bermuda'
 'Bolivia' 'Brazil' 'Barbados' 'Brunei' 'Bhutan' 'Botswana'
 'Central African Republic' 'Canada' 'Switzerland' 'Chile' 'China'
 "Cote d'Ivoire" 'Cameroon' 'Democratic Republic of Congo' 'Congo'
 'Colombia' 'Comoros' 'Cape Verde' 'Costa Rica' 'Cuba' 'Curacao'
 'Cayman Islands' 'Cyprus' 'Czech Republic' 'Germany' 'Djibouti'
 'Dominica' 'Denmark' 'Dominican Republic' 'Algeria' 'Ecuador' 'Egypt'
 'Eritrea' 'Western Sahara' 'Spain' 'Estonia' 'Ethiopia' 'Finland' 'Fiji'
 'Falkland Islands' 'France' 'Faeroe Islands' 'Gabon' 'United Kingdom'
 'Georgia' 'Guernsey' 'Ghana' 'Gibraltar' 'Guinea' 'Gambia'
 'Guinea-Bissau' 'Equatorial Guinea' 'Greece' 'Grenada' 

## 2. Covid 19 World Analysis - Most Recent Info

In [71]:
# Delete World Summary Figures
df.drop(df[df.location == 'World'].index, inplace=True)

# Getting the most recent date (max gives u the latest date)
df_most_recent = df[df['date'] == max(df['date'])].reset_index()
df_most_recent.head()

,index,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,219,ABW,North America,Aruba,2020-10-23,4389.0,20.0,14.857,36.0,1.0,0.571,41108.593,187.326,139.156,337.186,9.366,5.352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106766.0,584.800,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
1,517,AFG,Asia,Afghanistan,2020-10-23,40626.0,116.0,85.714,1505.0,4.0,3.429,1043.610,2.980,2.202,38.661,0.103,0.088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
2,733,AGO,Africa,Angola,2020-10-23,8338.0,289.0,213.143,255.0,4.0,4.000,253.695,8.793,6.485,7.759,0.122,0.122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32866268.0,23.890,16.8,2.405,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN,61.15,0.581
3,944,AIA,North America,Anguilla,2020-10-23,3.0,0.0,0.000,NaN,0.0,0.000,199.973,0.000,0.000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15002.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.88,NaN
4,1173,ALB,Europe,Albania,2020-10-23,18250.0,302.0,291.143,465.0,3.0,3.714,6341.650,104.941,101.169,161.582,1.042,1.291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2877800.0,104.871,38.0,13.188,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785


In [72]:
df_most_recent.tail()

,index,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
203,50717,YEM,Asia,Yemen,2020-10-23,2061.0,0.0,0.571,599.0,0.0,0.286,69.101,0.000,0.019,20.083,0.00,0.010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29825968.0,53.508,20.3,2.922,1.583,1479.147,18.8,495.003,5.35,7.6,29.2,49.542,0.70,66.12,0.452
204,50977,ZAF,Africa,South Africa,2020-10-23,710515.0,2156.0,1761.571,18843.0,102.0,76.286,11979.948,36.352,29.702,317.711,1.72,1.286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59308690.0,46.754,27.3,5.344,3.053,12294.876,18.9,200.380,5.52,8.1,33.2,43.993,2.32,64.13,0.699
205,51196,ZMB,Africa,Zambia,2020-10-23,16035.0,35.0,53.714,346.0,0.0,0.000,872.228,1.904,2.922,18.821,0.00,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18383956.0,22.995,17.7,2.480,1.542,3689.251,57.5,234.499,3.94,3.1,24.7,13.938,2.00,63.89,0.588
206,51413,ZWE,Africa,Zimbabwe,2020-10-23,8242.0,27.0,23.857,236.0,0.0,0.714,554.534,1.817,1.605,15.878,0.00,0.048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.70,61.49,0.535
207,52009,NaN,NaN,International,2020-10-23,696.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Creating a Summary Table

In [73]:
df_world = df_most_recent.groupby(["date"])[["total_cases","new_cases","new_deaths","total_deaths"]].sum().reset_index()

# Create Column headers/labels
labels = ["Last Update", "Total Confirmed","New Cases","New Deaths","Total Deaths"]

fig = go.Figure(data=[go.Table(header = dict(values=labels),
                 cells=dict(values=df_world.loc[0,["date","total_cases","new_cases","new_deaths","total_deaths"]]))
                     ])
fig.update_layout(title='Covid-19 World Summary: ',)
fig.show()

## 3. World Covid-19 Growth Over Time

In [74]:
df_over_time = df.groupby(["date"])[["total_cases","new_cases","total_deaths","new_deaths"]].sum().reset_index().sort_values("date",ascending=True).reset_index(drop=True)

df_over_time

,date,total_cases,new_cases,total_deaths,new_deaths
0,2019-12-31,27.0,27.0,0.0,0.0
1,2020-01-01,27.0,0.0,0.0,0.0
2,2020-01-02,27.0,0.0,0.0,0.0
3,2020-01-03,44.0,17.0,0.0,0.0
4,2020-01-04,44.0,0.0,0.0,0.0
...,...,...,...,...,...
293,2020-10-19,40168891.0,362206.0,1114898.0,3949.0
294,2020-10-20,40515637.0,346746.0,1119568.0,4670.0
295,2020-10-21,40901615.0,385978.0,1125969.0,6401.0
296,2020-10-22,41068608.0,434436.0,1122117.0,6688.0


In [75]:
df_over_time.drop(df_over_time[df_over_time.date == "2020-10-23"].index, inplace = True)
df_over_time

,date,total_cases,new_cases,total_deaths,new_deaths
0,2019-12-31,27.0,27.0,0.0,0.0
1,2020-01-01,27.0,0.0,0.0,0.0
2,2020-01-02,27.0,0.0,0.0,0.0
3,2020-01-03,44.0,17.0,0.0,0.0
4,2020-01-04,44.0,0.0,0.0,0.0
...,...,...,...,...,...
292,2020-10-18,39806685.0,368757.0,1110949.0,5538.0
293,2020-10-19,40168891.0,362206.0,1114898.0,3949.0
294,2020-10-20,40515637.0,346746.0,1119568.0,4670.0
295,2020-10-21,40901615.0,385978.0,1125969.0,6401.0


#### Confirmed Cases starting from day Zero (first covid case reported)

In [76]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_over_time.index, y=df_over_time['total_cases'],
                    mode='lines',
                    name='Confirmed cases'))


fig.update_layout(
    title='Evolution of Confirmed Covid-19 cases over time in the word',
        template='plotly_white',
      yaxis_title="Confirmed cases",
    xaxis_title="Days",

)

fig.show()

In [77]:
# Using a Data Axis Instead

fig = go.Figure(go.Bar(
            x=df_over_time['date'],
            y=df_over_time['total_cases'],
           ))
fig.update_layout(
    title='Confirmed Cases In Each Day',
    template='plotly_white',
     xaxis_title="Confirmed Cases",
    yaxis_title="Days",
)
fig.show()

In [78]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_over_time.index, y=df_over_time['total_deaths'],
                    mode='lines',marker_color='red',
                    name='Total Deaths',line=dict( dash='dot')))

fig.update_layout(
    title='Evolution of Covid-19 Deaths cases over time in the world',
        template='plotly_dark',
      yaxis_title="Number of Deaths",
    xaxis_title="Days",

)

fig.show()

In [79]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_over_time.date, y=df_over_time['new_cases'],
                    mode='lines',marker_color='green',
                    name='New Case Daily',line=dict( dash='dot')))

fig.update_layout(
    title='Evolution of Covid-19 New Cases over time in the world',
        template='plotly_dark',
      yaxis_title="New Cases",
    xaxis_title="Days",

)

fig.show()

In [80]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_over_time.date, y=df_over_time['new_deaths'],
                    mode='lines',marker_color='red',
                    name='New Case Daily',line=dict( dash='dot')))

fig.update_layout(
    title='Evolution of Covid-19 New Cases over time in the world',
        template='plotly_dark',
      yaxis_title="New Cases",
    xaxis_title="Days",

)

fig.show()

### 4. Confirmed Cases in Each Country

In [81]:
df_per_country = df.groupby(["location"])["new_cases","new_deaths"].sum().reset_index().sort_values("new_cases",ascending=False).reset_index(drop=True)
df_per_country.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,location,new_cases,new_deaths
0,United States,8408340.0,223042.0
1,India,7761312.0,117306.0
2,Brazil,5323630.0,155900.0
3,Russia,1463306.0,25242.0
4,Argentina,1053637.0,27957.0
5,Spain,1026281.0,34521.0
6,France,999043.0,34210.0
7,Colombia,990270.0,29636.0
8,Peru,879876.0,33984.0
9,Mexico,874171.0,87894.0


In [82]:
df_per_country.columns = ['Country', 'Total Cases', 'Total Deaths']
df_per_country = df_per_country.astype({"Country": str, "Total Cases": int, "Total Deaths":int})
df_per_country

,Country,Total Cases,Total Deaths
0,United States,8408340,223042
1,India,7761312,117306
2,Brazil,5323630,155900
3,Russia,1463306,25242
4,Argentina,1053637,27957
...,...,...,...
207,Montserrat,13,1
208,Falkland Islands,13,0
209,Anguilla,3,0
210,Solomon Islands,3,0


#### Let's display this as a barchart to see things in perspective

In [83]:
fig = go.Figure(go.Bar(
            x=df_per_country['Total Cases'],
            y=df_per_country['Country'],
            orientation='h'))
fig.update_layout(
    title='Confirmed Cases In Each Country',
    template='plotly_white',
     xaxis_title="Confirmed Cases",
    yaxis_title="Countries",
)
fig.show()

#### Let's do the same for Deaths

In [84]:
fig = go.Figure(go.Bar(
            x=df_per_country['Total Deaths'],
            y=df_per_country['Country'],
            orientation='h',
            marker_color='black',))
fig.update_layout(
    title='Deaths In Each Country',
    template='plotly_white',
    xaxis_title="Deaths",
    yaxis_title="Countries",
)

fig.show()

# 5. Maping Covid-19 Across the World

Let's look at total case numbers on a world map

In [85]:
fig = px.choropleth(df_per_country, locations=df_per_country['Country'],
                    color=df_per_country['Total Cases'],locationmode='country names', 
                    hover_name=df_per_country['Country'], 
                    color_continuous_scale=px.colors.sequential.OrRd,template='plotly_dark', )
fig.update_layout(
    title='Confirmed Cases In Each Country',
)
fig.show()

#### Adjusting for Population Sizes

In [86]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,ABW,North America,Aruba,2020-03-13,2.0,2.0,NaN,NaN,0.0,NaN,18.733,18.733,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
1,ABW,North America,Aruba,2020-03-19,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
2,ABW,North America,Aruba,2020-03-20,4.0,2.0,0.286,NaN,0.0,0.0,37.465,18.733,2.676,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
3,ABW,North America,Aruba,2020-03-21,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
4,ABW,North America,Aruba,2020-03-22,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN


# Let's group by country and get the total number of cases and deaths Per Million

In [87]:
df_per_country_pm = df.groupby(["location"])["new_cases_per_million","new_deaths_per_million"].sum().reset_index().sort_values("new_cases_per_million",ascending=False).reset_index(drop=True)
df_per_country_pm.columns = ['Country', 'Total Cases PM', 'Total Deaths PM']
df_per_country_pm = df_per_country_pm.astype({"Country": str, "Total Cases PM": int, "Total Deaths PM":int})
df_per_country_pm.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Country,Total Cases PM,Total Deaths PM
0,Andorra,49323,815
1,Bahrain,46551,181
2,Qatar,45282,79
3,Aruba,41108,337
4,Israel,35650,267


In [88]:
fig = px.choropleth(df_per_country_pm, locations=df_per_country_pm['Country'],
                    color=df_per_country_pm['Total Cases PM'],locationmode='country names', 
                    hover_name=df_per_country_pm['Country'], 
                    color_continuous_scale=px.colors.sequential.Viridis,template='plotly_dark', )
fig.update_layout(
    title='Confirmed Cases PM In Each Country',
)
fig.show()

#### Let's look at the top 10 countries per million on a Scatter Plot

In [89]:
fig = go.Figure(data=[go.Scatter(
    x=df_per_country_pm['Country'][0:10],
    y=df_per_country_pm['Total Cases PM'][0:10],
    mode='markers',
    marker=dict(
        color=100+np.random.randn(500),
        size=(df_per_country_pm['Total Cases PM'][0:10]/500), # Change 500 to larger as data evolves
        showscale=True
        )
)])
fig.update_layout(
    title='Most Infected Countries PM',
    xaxis_title="Countries",
    yaxis_title="Total Cases PM",
    template='plotly_white'

)
fig.show()

In [90]:
# Sort in descending order 
df_per_country_pm = df_per_country_pm.sort_values(by='Total Deaths PM', ascending = False)

fig = go.Figure(data=[go.Scatter(
    x=df_per_country_pm['Country'][0:10],
    y=df_per_country_pm['Total Deaths PM'][0:10],
    mode='markers',
    marker=dict(
        color=df_per_country_pm['Total Cases PM'],
        size=(df_per_country_pm['Total Deaths PM'][0:10]/10), # Change 10 to larger as data evolves
        showscale=True
        )
)])
fig.update_layout(
    title='Most Affected Countries by Deaths PM',
    xaxis_title="Countries",
    yaxis_title="Total Deaths PM",
    template='plotly_white'

)
fig.show()

# 6. Evolution of Confirmed Cases PM

In [92]:
df.tail()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
52005,NaN,NaN,International,2020-10-19,696.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52006,NaN,NaN,International,2020-10-20,696.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52007,NaN,NaN,International,2020-10-21,696.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52008,NaN,NaN,International,2020-10-22,696.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52009,NaN,NaN,International,2020-10-23,696.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
fig = px.choropleth(df, locations = df['location'],
                    color = df['total_cases'],locationmode='country names', 
                    hover_name = df['location'], 
                    color_continuous_scale = px.colors.sequential.Inferno,
                    animation_frame="date")
fig.update_layout(

    title='Evolution of Confirmed cases in each country',
    template='plotly_dark'
)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [93]:
fig = px.choropleth(df, locations = df['location'],
                    color = df['total_cases_per_million'],locationmode='country names', 
                    hover_name = df['location'], 
                    color_continuous_scale = px.colors.sequential.Inferno,
                    animation_frame="date")
fig.update_layout(

    title='Evolution of total cases per million in each country',
    template='plotly_dark'
)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

# 7. Comparing Continents & Countries

In [ ]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2019-12-31,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
1,AFG,Asia,Afghanistan,2020-01-01,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
2,AFG,Asia,Afghanistan,2020-01-02,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
3,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
4,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498


In [94]:
df_per_country_pm = df.groupby(["continent"])["new_cases_per_million","new_deaths_per_million"].sum().reset_index().sort_values("new_cases_per_million",ascending=False).reset_index(drop=True)
df_per_country_pm.columns = ['Country', 'Total Cases PM', 'Total Deaths PM']
df_per_country_pm = df_per_country_pm.astype({"Country": str, "Total Cases PM": int, "Total Deaths PM":int})
df_per_country_pm.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Country,Total Cases PM,Total Deaths PM
0,Europe,568709,12845
1,Asia,356711,3397
2,North America,287727,5962
3,South America,170809,5684
4,Africa,99446,1635



#### First let's compare USA vs the Rest of the World

In [95]:
df_usa = df[df['location'] == "United States"]
df_usa.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
48385,USA,North America,United States,2019-12-31,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924
48386,USA,North America,United States,2020-01-01,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924
48387,USA,North America,United States,2020-01-02,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924
48388,USA,North America,United States,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924
48389,USA,North America,United States,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924


In [96]:
url='https://covid.ourworldindata.org/data/owid-covid-data.csv'
df = pd.read_csv(url)

df_world = df[df['location'] == "World"]
df_world.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
51414,OWID_WRL,NaN,World,2019-12-31,27.0,27.0,NaN,NaN,0.0,NaN,0.003,0.003,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.794799e+09,58.045,30.9,8.696,5.355,15469.207,10.0,233.07,8.51,6.434,34.635,60.13,2.705,72.58,NaN
51415,OWID_WRL,NaN,World,2020-01-01,27.0,0.0,NaN,NaN,0.0,NaN,0.003,0.000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.794799e+09,58.045,30.9,8.696,5.355,15469.207,10.0,233.07,8.51,6.434,34.635,60.13,2.705,72.58,NaN
51416,OWID_WRL,NaN,World,2020-01-02,27.0,0.0,NaN,NaN,0.0,NaN,0.003,0.000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.794799e+09,58.045,30.9,8.696,5.355,15469.207,10.0,233.07,8.51,6.434,34.635,60.13,2.705,72.58,NaN
51417,OWID_WRL,NaN,World,2020-01-03,44.0,17.0,NaN,NaN,0.0,NaN,0.006,0.002,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.794799e+09,58.045,30.9,8.696,5.355,15469.207,10.0,233.07,8.51,6.434,34.635,60.13,2.705,72.58,NaN
51418,OWID_WRL,NaN,World,2020-01-04,44.0,0.0,NaN,NaN,0.0,NaN,0.006,0.000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.794799e+09,58.045,30.9,8.696,5.355,15469.207,10.0,233.07,8.51,6.434,34.635,60.13,2.705,72.58,NaN


In [98]:
df_india = df[df['location'] == "India"]
df_india.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
22185,IND,Asia,India,2019-12-31,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.380004e+09,450.419,28.2,5.989,3.414,6426.674,21.2,282.28,10.39,1.9,20.6,59.55,0.53,69.66,0.64
22186,IND,Asia,India,2020-01-01,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.380004e+09,450.419,28.2,5.989,3.414,6426.674,21.2,282.28,10.39,1.9,20.6,59.55,0.53,69.66,0.64
22187,IND,Asia,India,2020-01-02,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.380004e+09,450.419,28.2,5.989,3.414,6426.674,21.2,282.28,10.39,1.9,20.6,59.55,0.53,69.66,0.64
22188,IND,Asia,India,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.380004e+09,450.419,28.2,5.989,3.414,6426.674,21.2,282.28,10.39,1.9,20.6,59.55,0.53,69.66,0.64
22189,IND,Asia,India,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.380004e+09,450.419,28.2,5.989,3.414,6426.674,21.2,282.28,10.39,1.9,20.6,59.55,0.53,69.66,0.64


In [99]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = df_usa['date'], y = df_usa['total_cases'],
                    mode='lines',
                    name='Confirmed cases in US'))

fig.add_trace(go.Scatter(x = df_india['date'], y = df_india['total_cases'],
                    mode='lines',
                    name='Confirmed cases in India'))

fig.add_trace(go.Scatter(x = df_world['date'], y = df_world['total_cases'],
                    mode='lines',
                    name='Confirmed cases in The Rest Of The Word'))

fig.update_layout(
    title='Evolution of Confirmed cases over time in US and The Rest Of The Word',
        template='plotly_white'

)

fig.show()

#### Let's Compare Deaths in the US vs the Rest of the World

In [100]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = df_usa['date'], y = df_usa['total_deaths'],
                    mode='lines',
                    name='Confirmed Deaths in US'))


fig.add_trace(go.Scatter(x = df_world['date'], y = df_world['total_deaths'],
                    mode='lines',
                    name='Confirmed Deaths in The Rest Of The Word'))

fig.update_layout(
    title='Evolution of Confirmed Deaths over time in US and The Rest Of The Word',
        template='plotly_white'

)

fig.show()

# 8. Let's create a dataset we can import into Flourish

<center><img src="https://i.imgur.com/v3fiqrx.png"></center>

### Let's reload our data

In [2]:
import pandas as pd

url='https://covid.ourworldindata.org/data/owid-covid-data.csv'
df = pd.read_csv(url)
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2019-12-31,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
1,AFG,Asia,Afghanistan,2020-01-01,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
2,AFG,Asia,Afghanistan,2020-01-02,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
3,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
4,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498


#### Now let's change the time format


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52222 entries, 0 to 52221
Data columns (total 41 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   iso_code                         51923 non-null  object 
 1   continent                        51624 non-null  object 
 2   location                         52222 non-null  object 
 3   date                             52222 non-null  object 
 4   total_cases                      48615 non-null  float64
 5   new_cases                        51338 non-null  float64
 6   new_cases_smoothed               50550 non-null  float64
 7   total_deaths                     39750 non-null  float64
 8   new_deaths                       51338 non-null  float64
 9   new_deaths_smoothed              50550 non-null  float64
 10  total_cases_per_million          48352 non-null  float64
 11  new_cases_per_million            51274 non-null  float64
 12  new_cases_smoothed

In [4]:
# Make data column a pandas datetime data type
df['date'] = pd.to_datetime(df.date)

# Change format to MM/DD/YY
df['date'] = df['date'].dt.strftime('%m/%d/%y')
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,12/31/19,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
1,AFG,Asia,Afghanistan,01/01/20,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
2,AFG,Asia,Afghanistan,01/02/20,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
3,AFG,Asia,Afghanistan,01/03/20,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
4,AFG,Asia,Afghanistan,01/04/20,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498


In [5]:
# Set index to date
df = df.set_index('date')
df.head()

,iso_code,continent,location,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12/31/19,AFG,Asia,Afghanistan,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
01/01/20,AFG,Asia,Afghanistan,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
01/02/20,AFG,Asia,Afghanistan,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
01/03/20,AFG,Asia,Afghanistan,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
01/04/20,AFG,Asia,Afghanistan,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498


#### Now let's change the data format and get it ready for Flourish

In [6]:
df_florish = pd.pivot_table(df, values='total_cases', 
                     index=['location'], 
                     columns='date').reset_index()

df_florish = df_florish.fillna(0)

df_florish.head()

date,location,01/01/20,01/02/20,01/03/20,01/04/20,01/05/20,01/06/20,01/07/20,01/08/20,01/09/20,01/10/20,01/11/20,01/12/20,01/13/20,01/14/20,01/15/20,01/16/20,01/17/20,01/18/20,01/19/20,01/20/20,01/21/20,01/22/20,01/23/20,01/24/20,01/25/20,01/26/20,01/27/20,01/28/20,01/29/20,01/30/20,01/31/20,02/01/20,02/02/20,02/03/20,02/04/20,02/05/20,02/06/20,02/07/20,02/08/20,...,09/16/20,09/17/20,09/18/20,09/19/20,09/20/20,09/21/20,09/22/20,09/23/20,09/24/20,09/25/20,09/26/20,09/27/20,09/28/20,09/29/20,09/30/20,10/01/20,10/02/20,10/03/20,10/04/20,10/05/20,10/06/20,10/07/20,10/08/20,10/09/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,12/31/19
0,Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38855.0,38872.0,38872.0,38919.0,39044.0,39044.0,39074.0,39145.0,39170.0,39186.0,39192.0,39227.0,39227.0,39239.0,39254.0,39268.0,39285.0,39290.0,39297.0,39341.0,39486.0,39548.0,39616.0,39693.0,39703.0,39703.0,39799.0,39928.0,39994.0,40026.0,40026.0,40073.0,40141.0,40200.0,40287.0,40375.0,40510.0,40626.0,40687.0,0.0
1,Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11672.0,11816.0,11948.0,12073.0,12226.0,12385.0,12535.0,12666.0,12787.0,12921.0,13045.0,13153.0,13259.0,13391.0,13518.0,13649.0,13806.0,13965.0,14117.0,14266.0,14410.0,14568.0,14730.0,14899.0,15066.0,15231.0,15399.0,15570.0,15752.0,15955.0,16212.0,16501.0,16774.0,17055.0,17350.0,17651.0,17948.0,18250.0,18556.0,0.0
2,Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48734.0,48966.0,49194.0,49413.0,49623.0,49826.0,50023.0,50214.0,50400.0,50579.0,50754.0,50914.0,51067.0,51213.0,51368.0,51530.0,51690.0,51847.0,51995.0,52136.0,52270.0,52399.0,52520.0,52658.0,52804.0,52940.0,53072.0,53225.0,53399.0,53584.0,53777.0,53988.0,54203.0,54402.0,54616.0,54839.0,55091.0,55357.0,55630.0,0.0
3,Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1438.0,1438.0,1483.0,1564.0,1564.0,1564.0,1681.0,1681.0,1753.0,1753.0,1836.0,1836.0,1836.0,1966.0,1966.0,2050.0,2050.0,2110.0,2110.0,2110.0,2370.0,2370.0,2568.0,2568.0,2696.0,2696.0,2696.0,2995.0,2995.0,3190.0,3190.0,3377.0,3377.0,3377.0,3623.0,3623.0,3811.0,3811.0,4038.0,0.0
4,Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3439.0,3569.0,3749.0,3789.0,3901.0,3991.0,4117.0,4236.0,4236.0,4363.0,4475.0,4590.0,4672.0,4797.0,4905.0,4972.0,4972.0,5114.0,5211.0,5370.0,5370.0,5725.0,5725.0,5863.0,5958.0,6031.0,6246.0,6366.0,6488.0,6680.0,6846.0,7096.0,7222.0,7462.0,7622.0,7829.0,8049.0,8338.0,8582.0,0.0


### We want flags in our plot

In [8]:
file_name = "https://raw.githubusercontent.com/rajeevratan84/covid/master/flags_world.csv"
flags = pd.read_csv(file_name)
flags

,country,Image URL
0,Afghanistan,https://upload.wikimedia.org/wikipedia/commons...
1,Angola,https://cdn.countryflags.com/thumbs/angola/fla...
2,Albania,https://cdn.countryflags.com/thumbs/albania/fl...
3,Andorra,https://cdn.countryflags.com/thumbs/andorra/fl...
4,Argentina,https://cdn.countryflags.com/thumbs/argentina/...
...,...,...
203,Kosovo,https://cdn.countryflags.com/thumbs/kosovo/fla...
204,"Yemen, Rep.",https://flagpedia.net/data/flags/w580/ye.png
205,South Africa,https://cdn.countryflags.com/thumbs/south-afri...
206,Zambia,https://cdn.countryflags.com/thumbs/zambia/fla...


## Merging out data with the flag URL dataset

In [9]:
df_florish = pd.merge(df_florish, flags, left_on='location', right_on='country')
df_florish.head()

,location,01/01/20,01/02/20,01/03/20,01/04/20,01/05/20,01/06/20,01/07/20,01/08/20,01/09/20,01/10/20,01/11/20,01/12/20,01/13/20,01/14/20,01/15/20,01/16/20,01/17/20,01/18/20,01/19/20,01/20/20,01/21/20,01/22/20,01/23/20,01/24/20,01/25/20,01/26/20,01/27/20,01/28/20,01/29/20,01/30/20,01/31/20,02/01/20,02/02/20,02/03/20,02/04/20,02/05/20,02/06/20,02/07/20,02/08/20,...,09/18/20,09/19/20,09/20/20,09/21/20,09/22/20,09/23/20,09/24/20,09/25/20,09/26/20,09/27/20,09/28/20,09/29/20,09/30/20,10/01/20,10/02/20,10/03/20,10/04/20,10/05/20,10/06/20,10/07/20,10/08/20,10/09/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,12/31/19,country,Image URL
0,Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38872.0,38919.0,39044.0,39044.0,39074.0,39145.0,39170.0,39186.0,39192.0,39227.0,39227.0,39239.0,39254.0,39268.0,39285.0,39290.0,39297.0,39341.0,39486.0,39548.0,39616.0,39693.0,39703.0,39703.0,39799.0,39928.0,39994.0,40026.0,40026.0,40073.0,40141.0,40200.0,40287.0,40375.0,40510.0,40626.0,40687.0,0.0,Afghanistan,https://upload.wikimedia.org/wikipedia/commons...
1,Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11948.0,12073.0,12226.0,12385.0,12535.0,12666.0,12787.0,12921.0,13045.0,13153.0,13259.0,13391.0,13518.0,13649.0,13806.0,13965.0,14117.0,14266.0,14410.0,14568.0,14730.0,14899.0,15066.0,15231.0,15399.0,15570.0,15752.0,15955.0,16212.0,16501.0,16774.0,17055.0,17350.0,17651.0,17948.0,18250.0,18556.0,0.0,Albania,https://cdn.countryflags.com/thumbs/albania/fl...
2,Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49194.0,49413.0,49623.0,49826.0,50023.0,50214.0,50400.0,50579.0,50754.0,50914.0,51067.0,51213.0,51368.0,51530.0,51690.0,51847.0,51995.0,52136.0,52270.0,52399.0,52520.0,52658.0,52804.0,52940.0,53072.0,53225.0,53399.0,53584.0,53777.0,53988.0,54203.0,54402.0,54616.0,54839.0,55091.0,55357.0,55630.0,0.0,Algeria,https://cdn.countryflags.com/thumbs/algeria/fl...
3,Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1483.0,1564.0,1564.0,1564.0,1681.0,1681.0,1753.0,1753.0,1836.0,1836.0,1836.0,1966.0,1966.0,2050.0,2050.0,2110.0,2110.0,2110.0,2370.0,2370.0,2568.0,2568.0,2696.0,2696.0,2696.0,2995.0,2995.0,3190.0,3190.0,3377.0,3377.0,3377.0,3623.0,3623.0,3811.0,3811.0,4038.0,0.0,Andorra,https://cdn.countryflags.com/thumbs/andorra/fl...
4,Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3749.0,3789.0,3901.0,3991.0,4117.0,4236.0,4236.0,4363.0,4475.0,4590.0,4672.0,4797.0,4905.0,4972.0,4972.0,5114.0,5211.0,5370.0,5370.0,5725.0,5725.0,5863.0,5958.0,6031.0,6246.0,6366.0,6488.0,6680.0,6846.0,7096.0,7222.0,7462.0,7622.0,7829.0,8049.0,8338.0,8582.0,0.0,Angola,https://cdn.countryflags.com/thumbs/angola/fla...


### Generating our final file for Flourish

In [12]:
from datetime import datetime

# Use current date for timestamped filename
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d_%b_%Y")

# Delete unnecessary columns
del df_florish['country']
del df_florish['12/31/19']

# Create a series to hold the Image URLS
mid = df_florish['Image URL']

# Delete the existing Image URLs
df_florish.drop(labels=['Image URL'], axis=1, inplace = True)

# Insert Mid into the second position in the dataframe
df_florish.insert(1, 'Image URL', mid)

# Give the CSV a name with the current timestamp
csv_name = 'world_covid_cases_' + timestampStr + '.csv'
df_florish.to_csv(csv_name)
df_florish.head()

,location,Image URL,01/01/20,01/02/20,01/03/20,01/04/20,01/05/20,01/06/20,01/07/20,01/08/20,01/09/20,01/10/20,01/11/20,01/12/20,01/13/20,01/14/20,01/15/20,01/16/20,01/17/20,01/18/20,01/19/20,01/20/20,01/21/20,01/22/20,01/23/20,01/24/20,01/25/20,01/26/20,01/27/20,01/28/20,01/29/20,01/30/20,01/31/20,02/01/20,02/02/20,02/03/20,02/04/20,02/05/20,02/06/20,02/07/20,...,09/15/20,09/16/20,09/17/20,09/18/20,09/19/20,09/20/20,09/21/20,09/22/20,09/23/20,09/24/20,09/25/20,09/26/20,09/27/20,09/28/20,09/29/20,09/30/20,10/01/20,10/02/20,10/03/20,10/04/20,10/05/20,10/06/20,10/07/20,10/08/20,10/09/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20
0,Afghanistan,https://upload.wikimedia.org/wikipedia/commons...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38815.0,38855.0,38872.0,38872.0,38919.0,39044.0,39044.0,39074.0,39145.0,39170.0,39186.0,39192.0,39227.0,39227.0,39239.0,39254.0,39268.0,39285.0,39290.0,39297.0,39341.0,39486.0,39548.0,39616.0,39693.0,39703.0,39703.0,39799.0,39928.0,39994.0,40026.0,40026.0,40073.0,40141.0,40200.0,40287.0,40375.0,40510.0,40626.0,40687.0
1,Albania,https://cdn.countryflags.com/thumbs/albania/fl...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11520.0,11672.0,11816.0,11948.0,12073.0,12226.0,12385.0,12535.0,12666.0,12787.0,12921.0,13045.0,13153.0,13259.0,13391.0,13518.0,13649.0,13806.0,13965.0,14117.0,14266.0,14410.0,14568.0,14730.0,14899.0,15066.0,15231.0,15399.0,15570.0,15752.0,15955.0,16212.0,16501.0,16774.0,17055.0,17350.0,17651.0,17948.0,18250.0,18556.0
2,Algeria,https://cdn.countryflags.com/thumbs/algeria/fl...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48496.0,48734.0,48966.0,49194.0,49413.0,49623.0,49826.0,50023.0,50214.0,50400.0,50579.0,50754.0,50914.0,51067.0,51213.0,51368.0,51530.0,51690.0,51847.0,51995.0,52136.0,52270.0,52399.0,52520.0,52658.0,52804.0,52940.0,53072.0,53225.0,53399.0,53584.0,53777.0,53988.0,54203.0,54402.0,54616.0,54839.0,55091.0,55357.0,55630.0
3,Andorra,https://cdn.countryflags.com/thumbs/andorra/fl...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1438.0,1438.0,1438.0,1483.0,1564.0,1564.0,1564.0,1681.0,1681.0,1753.0,1753.0,1836.0,1836.0,1836.0,1966.0,1966.0,2050.0,2050.0,2110.0,2110.0,2110.0,2370.0,2370.0,2568.0,2568.0,2696.0,2696.0,2696.0,2995.0,2995.0,3190.0,3190.0,3377.0,3377.0,3377.0,3623.0,3623.0,3811.0,3811.0,4038.0
4,Angola,https://cdn.countryflags.com/thumbs/angola/fla...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3388.0,3439.0,3569.0,3749.0,3789.0,3901.0,3991.0,4117.0,4236.0,4236.0,4363.0,4475.0,4590.0,4672.0,4797.0,4905.0,4972.0,4972.0,5114.0,5211.0,5370.0,5370.0,5725.0,5725.0,5863.0,5958.0,6031.0,6246.0,6366.0,6488.0,6680.0,6846.0,7096.0,7222.0,7462.0,7622.0,7829.0,8049.0,8338.0,8582.0


## Quick setup tips for Flourish!

**When inside Flourish, Under Data:**
1. Set Labels to B
2. Set Categories Blank
3. Label B
4. Image C

**In Flourish, Under Preview:**
1. In Bars set No. Bars to 20
2. In Bar colors, set by Bar
3. In Timeline and Animation turn off Loop Timeline
4. In Controls add Sort Control (Optional)
5. Number Formatting Add comma
6. Add Title in Header